<a href="https://colab.research.google.com/github/shinchacoffee/CIFAR10_practice/blob/main/CIFAR10_test_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision

In [2]:
random_seed=1
train_batch_size=100
test_batch_size=1000
n_epochs=1
torch.manual_seed(random_seed)

In [3]:
train_norm_load = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR10('/files/', train = True,
                                 transform = torchvision.transforms.Compose([
                                            torchvision.transforms.ToTensor()
                                            ]),
                                            download = True),
                            batch_size=train_batch_size, shuffle = True)

test_norm_load = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR10('/files/', train = False,
                                 transform = torchvision.transforms.Compose([
                                            torchvision.transforms.ToTensor()
                                            ]),
                                            download = True),
                            batch_size=train_batch_size, shuffle = True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /files/cifar-10-python.tar.gz to /files/
Files already downloaded and verified


In [4]:
RGB_sum = torch.zeros(3)
RGB_sum_sq = torch.zeros(3)
for x,y in train_norm_load:
  RGB_sum += torch.mean(x, dim=(0,2,3))#*x.shape[0]
  RGB_sum_sq += torch.mean(x*x, dim=(0,2,3))#*x.shape[0]
RGB_mean = RGB_sum / len(train_norm_load)
RGB_std  = (RGB_sum_sq / len(train_norm_load) - RGB_mean ** 2)**0.5
print(RGB_mean,RGB_std)

test_RGB_sum = torch.zeros(3)
test_RGB_sum_sq = torch.zeros(3)
for x,y in test_norm_load:
  test_RGB_sum += torch.mean(x, dim=(0,2,3))#*x.shape[0]
  test_RGB_sum_sq += torch.mean(x*x, dim=(0,2,3))#*x.shape[0]
test_RGB_mean = test_RGB_sum / len(test_norm_load)
test_RGB_std  = (test_RGB_sum_sq / len(test_norm_load) - test_RGB_mean ** 2)**0.5
print(test_RGB_mean,test_RGB_std)

tensor([0.4914, 0.4822, 0.4465]) tensor([0.2470, 0.2435, 0.2616])
tensor([0.4942, 0.4851, 0.4504]) tensor([0.2467, 0.2429, 0.2616])


In [5]:
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR10('/files/', train = True,
                                 transform = torchvision.transforms.Compose([
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize(
                                                RGB_mean, RGB_std )
                                            ]),
                                            download = True),
                            batch_size=train_batch_size, shuffle = True)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.CIFAR10('/files/', train = False,
                                 transform = torchvision.transforms.Compose([
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize(
                                                test_RGB_mean, test_RGB_std )
                                            ]), #why not working when without enter?
                                            download = True),
                            batch_size=test_batch_size, shuffle = True)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
import torch.nn
import torch.nn.functional
import torch.optim

In [71]:
class My_net(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.cnum1, self.cdim1, self.cnum2, self.cdim2 = int(30), int(9), int(90), int(9)
    self.linnum2 = int(40)
    # 15 na 45 i dawalo 60% po 10ep (drugie okno 7? a pierwsze 5?7?)
    # 30/7? na 90/9 i dawalo 65% po 10ep (10 min?)
    # 50/9 na 150/7 i dawalo 67% po 10ep (20 min?)
    # 30/9 na 300/11 i dawalo 66% po 10ep (12 min)
    # 50/9 na 200/11 i dawalo 65% po 10 ep (17 min)
    # 10/1 na 300/15 i dawalo 52% po 10 ep (8 min)
    # 30/9 na 90/9 i dawalo 64% po 10ep (9 min) - lr=0.005 mom=0.9 wdec=0.001
    # 30/9 na 90/9 i dawalo 57% po 10ep (9 min) - lr=0.01 mom=0.9 wdec=0.01
    # 30/9 na 90/9 i dawalo 62% po 10ep (9 min) - lr=0.003 mom=0.73 wdec=0.0
    self.conv1 = torch.nn.Conv2d(3,self.cnum1,self.cdim1)
    self.conv2 = torch.nn.Conv2d(self.cnum1,self.cnum2,self.cdim2)
    self.conv2drop = torch.nn.Dropout2d(p=0.5)        
    self.skip_conv2 = torch.nn.Conv2d(3,15,13) 
    self.lin_conv_num = int( ((33 - self.cdim1) / 2 - self.cdim2 + 1) * ((33 - self.cdim1) / 2 - self.cdim2 + 1) * self.cnum2 / 4 ) 
    self.line1 = torch.nn.Linear(self.lin_conv_num,self.linnum2)
    self.line2 = torch.nn.Linear(self.linnum2,10)
    self.resid = torch.nn.Linear(self.lin_conv_num,10)
    print(self.lin_conv_num)

  def forward(self, signal):
    #print(signal.shape)
    #singal = torch.nn.BatchNorm2d(3)(signal)
    #residconv = torch.nn.functional.max_pool2d(self.skip_conv2(signal),2)
    #print(residconv.shape)
    signal = torch.nn.functional.relu(torch.nn.functional.max_pool2d(self.conv1(signal),2))
    #print(signal.shape)
    #signal = torch.nn.BatchNorm2d(9)(signal)
    #+residconv+residconv
    signal = torch.nn.functional.relu(torch.nn.functional.max_pool2d(self.conv2drop(self.conv2(signal)),2))
    #print(signal.shape)
    signal = torch.nn.BatchNorm2d(self.cnum2)(signal)
    signal = signal.view(signal.shape[0],-1)
    #print(signal.shape)
    residual = torch.nn.functional.relu(self.resid(signal))
    signal = torch.nn.functional.relu(self.line1(signal))
    signal = torch.nn.LayerNorm(self.linnum2)(signal)
    signal = torch.nn.functional.dropout(signal,p=0.5)
    #print(signal.shape)
    #signal += residual
    signal = torch.nn.functional.relu(self.line2(signal)+residual)#
    #print(signal.shape)
    return torch.nn.functional.log_softmax(signal)

sieczioszka = My_net()

360


In [8]:
train_losses = []
train_counter = []
def train(network, optimizer, epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()

    #rotator = torchvision.transforms.RandomRotation(degrees=(-10, 10))
    #data = rotator(data)
    #prob=torch.rand(1)
    #if prob[0] > 0.5:
    #  data = torchvision.transforms.Grayscale(num_output_channels=3)(data)
    my_transforms = torch.nn.ModuleList([#torchvision.transforms.ColorJitter(), #useless cause by default is zero
                                         #torchvision.transforms.RandomGrayscale(p=0.2),
                                         #torchvision.transforms.RandomRotation(degrees=(-10, 10)),
                                         #torchvision.transforms.RandomPerspective(),
                                         torchvision.transforms.RandomVerticalFlip(p=0.5)#,
                                         #torchvision.transforms.GaussianBlur(3, sigma=(0.1, 1.0)),
                                         ])
    randtran = torchvision.transforms.RandomApply(transforms=my_transforms, p=0.007)
    data = randtran(data)

    output = network(data)
    loss = torch.nn.functional.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 250 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))


In [9]:
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]
def test(network):
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += torch.nn.functional.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [76]:
network = My_net()
learnin_rate = 0.003
my_momentum = 0.73
my_weight_decay = 0.0
optimizer = torch.optim.SGD(network.parameters(), lr=learnin_rate, momentum=my_momentum, weight_decay=my_weight_decay)

360


In [36]:
network(torch.rand(7,3,32,32))

torch.Size([7, 1350])


RuntimeError: ignored

In [77]:
n_epochs=10
for epoch in range(1, n_epochs+1):
  train(network, optimizer, epoch)

test(network)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.446067
Train Epoch: 1 [25000/50000 (50%)]	Loss: 1.763200
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.550665
Train Epoch: 2 [25000/50000 (50%)]	Loss: 1.634256
Train Epoch: 3 [0/50000 (0%)]	Loss: 1.707480
Train Epoch: 3 [25000/50000 (50%)]	Loss: 1.623995
Train Epoch: 4 [0/50000 (0%)]	Loss: 1.382026
Train Epoch: 4 [25000/50000 (50%)]	Loss: 1.503168
Train Epoch: 5 [0/50000 (0%)]	Loss: 1.234679
Train Epoch: 5 [25000/50000 (50%)]	Loss: 1.315667
Train Epoch: 6 [0/50000 (0%)]	Loss: 1.340116
Train Epoch: 6 [25000/50000 (50%)]	Loss: 1.245095
Train Epoch: 7 [0/50000 (0%)]	Loss: 1.071172
Train Epoch: 7 [25000/50000 (50%)]	Loss: 1.356428
Train Epoch: 8 [0/50000 (0%)]	Loss: 1.139522
Train Epoch: 8 [25000/50000 (50%)]	Loss: 1.402444
Train Epoch: 9 [0/50000 (0%)]	Loss: 1.146772
Train Epoch: 9 [25000/50000 (50%)]	Loss: 1.247239
Train Epoch: 10 [0/50000 (0%)]	Loss: 1.035042
Train Epoch: 10 [25000/50000 (50%)]	Loss: 1.173590


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 1.0943, Accuracy: 6245/10000 (62%)



In [ ]:
n_epochs=10
for epoch in range(1, n_epochs+1):
  train(network, optimizer, epoch)

test(network)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/50000 (0%)]	Loss: 1.016146
Train Epoch: 1 [25000/50000 (50%)]	Loss: 1.125848
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.079475
Train Epoch: 2 [25000/50000 (50%)]	Loss: 1.025132
Train Epoch: 3 [0/50000 (0%)]	Loss: 1.043578
Train Epoch: 3 [25000/50000 (50%)]	Loss: 1.367377
Train Epoch: 4 [0/50000 (0%)]	Loss: 1.067210
Train Epoch: 4 [25000/50000 (50%)]	Loss: 1.111030
Train Epoch: 5 [0/50000 (0%)]	Loss: 0.949393
Train Epoch: 5 [25000/50000 (50%)]	Loss: 1.167679
Train Epoch: 6 [0/50000 (0%)]	Loss: 1.119221
Train Epoch: 6 [25000/50000 (50%)]	Loss: 0.937928
Train Epoch: 7 [0/50000 (0%)]	Loss: 0.916493
Train Epoch: 7 [25000/50000 (50%)]	Loss: 1.054989
Train Epoch: 8 [0/50000 (0%)]	Loss: 0.854970
Train Epoch: 8 [25000/50000 (50%)]	Loss: 1.086734
Train Epoch: 9 [0/50000 (0%)]	Loss: 1.003292
Train Epoch: 9 [25000/50000 (50%)]	Loss: 1.139560
Train Epoch: 10 [0/50000 (0%)]	Loss: 1.032081
Train Epoch: 10 [25000/50000 (50%)]	Loss: 1.033481


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.9565, Accuracy: 6798/10000 (68%)

